# Personal Project main notebook

In [1]:
# Ignoring warning messages from python
import warnings
warnings.filterwarnings('ignore')

# General use imports
import pandas as pd
import numpy as np

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Modules and data
import requests
import acquire
import prep
from vega_datasets import data
from datetime import datetime

# I. Acquire

In [2]:
# Acquiring the data from the csv file and displaying first 5 rows

transactions1 = acquire.get_local_transactions()
transactions1.head()

,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
0,6/29/2022,DEPOSIT@MOBILE,DEPOSIT@MOBILE,206.25,credit,Transfer,Checkarama,NaN,NaN
1,6/28/2022,"MATTHEWS VET SERVICES, P","MATTHEWS VET SERVICES, P",13.20,debit,Veterinary,CREDIT CARD,NaN,NaN
2,6/28/2022,STARBUCKS STORE 24306,STARBUCKS STORE 24306,5.57,debit,Coffee Shops,CREDIT CARD,NaN,NaN
3,6/28/2022,PAYPAL *HULU,PAYPAL *HULU,14.16,debit,Television,CREDIT CARD,NaN,NaN
4,6/28/2022,H-E-B #618,H-E-B #618,20.56,debit,Groceries,CREDIT CARD,NaN,NaN


# II. Clean & Prepare

>## High level exploration before Cleaning and preparation

In [3]:
# Checking the number of rows and columns

transactions1.shape

(7872, 9)

In [4]:
# Comparing the Description and the Original Description columns to see which is worth dropping

transactions1[['Description', 'Original Description']]

,Description,Original Description
0,DEPOSIT@MOBILE,DEPOSIT@MOBILE
1,"MATTHEWS VET SERVICES, P","MATTHEWS VET SERVICES, P"
2,STARBUCKS STORE 24306,STARBUCKS STORE 24306
3,PAYPAL *HULU,PAYPAL *HULU
4,H-E-B #618,H-E-B #618
...,...,...
7867,Ultra Foods Qps,ULTRA FOODS #8761 QPS LOMBARD
7868,Pennys Noodle Shop,PENNY'S NOODLE SHOPQPS CHICAGO
7869,Amazon Music,MUSIC DWNLDS 866-216
7870,Calling Card,CALLINGCARDS/CONFERENC (866)29


In [5]:
# Checking the df nulls

transactions1.isnull().sum()

Date                       0
Description                0
Original Description       0
Amount                     0
Transaction Type           0
Category                   3
Account Name               0
Labels                  7870
Notes                   7862
dtype: int64

In [6]:
# Checking the specific crows of the Category column

transactions1[transactions1['Category'].isnull()]

,Date,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
5996,3/02/2015,Payment,Payment,10.0,credit,NaN,Stafford Loans U.S. DEPARTMENT OF EDUCATION,NaN,NaN
5997,3/02/2015,Pending,Pending,10.0,credit,NaN,Stafford Loans U.S. DEPARTMENT OF EDUCATION,NaN,NaN
5998,2/28/2015,Payment,Payment,40.0,credit,NaN,Stafford Loans U.S. DEPARTMENT OF EDUCATION,NaN,NaN


>## Takeaways
    - The df contains duplicates
    - The df has nulls in great number in labels and Notes columns
    - The df has much fewer nulls in the Category column
    - The Description and the Original Description are duplicates
    - These columns contain special characters
    - There is a date column
>## Actions
    - Remove duplicates
    - Drop Labels, Notes, and Original Description
    - Remove special characters from Description
    - Rename columns for readability
    - Make Account Name and Description's content into lower case
    - Set Date as INDEX

>## Cleaning

In [7]:
# Printing the column names into a list without a line break

a = transactions1.columns.tolist()
print(a,)

['Date', 'Description', 'Original Description', 'Amount', 'Transaction Type', 'Category', 'Account Name', 'Labels', 'Notes']


In [8]:
# Assigning the list of columns to a variable

columns = ['Date', 'Description', 'Original Description', 'Amount', 'Transaction Type', 'Category', 'Account Name', 'Labels', 'Notes']

In [10]:
# Displaying duplicate rows in count and percent

prep.multi_frequency(transactions1, columns)

,num_rows_missing,pct_rows_missing
Date,0,0.000000
Description,0,0.000000
Original Description,0,0.000000
Amount,0,0.000000
Transaction Type,0,0.000000
Category,3,0.038110
Account Name,0,0.000000
Labels,7870,99.974593
Notes,7862,99.872967


In [12]:
# Dropping dupicates and keeping the entries detected and reassigning the df to a new variable

transactions2 = transactions1.drop_duplicates()

In [13]:
transactions2.shape

(7797, 9)

In [14]:
# Dropping columns that are not useful

transactions3 = transactions2.drop(['Account Name', 'Labels', 'Notes', 'Original Description'], axis = 1)

In [15]:
transactions3.head()

,Date,Description,Amount,Transaction Type,Category
0,6/29/2022,DEPOSIT@MOBILE,206.25,credit,Transfer
1,6/28/2022,"MATTHEWS VET SERVICES, P",13.20,debit,Veterinary
2,6/28/2022,STARBUCKS STORE 24306,5.57,debit,Coffee Shops
3,6/28/2022,PAYPAL *HULU,14.16,debit,Television
4,6/28/2022,H-E-B #618,20.56,debit,Groceries


In [16]:
# Checking the nulls in the Category column

transactions3[transactions3['Category'].isnull()]

,Date,Description,Amount,Transaction Type,Category
5996,3/02/2015,Payment,10.0,credit,NaN
5997,3/02/2015,Pending,10.0,credit,NaN
5998,2/28/2015,Payment,40.0,credit,NaN


In [17]:
# Filling out the nulls with the category 'Education'

transactions4 = prep.handle_missing_values(transactions3)

In [18]:
a = transactions4.columns.tolist()
print(a,)

['Date', 'Description', 'Amount', 'Transaction Type', 'Category']


In [19]:
# Assigning columns to a variable

columns4 = ['Date', 'Description', 'Amount', 'Transaction Type', 'Category']

In [20]:
# Checking nulls

prep.multi_frequency(transactions4, columns4)

,num_rows_missing,pct_rows_missing
Date,0,0.0
Description,0,0.0
Amount,0,0.0
Transaction Type,0,0.0
Category,0,0.0


In [22]:
# Checking the number of unique entries in Description column

transactions4['Description'].nunique()

1714

In [23]:
# Checking the number of unique entries in Transaction Type column (Debit and Credit)

transactions4['Transaction Type'].nunique()

2

In [24]:
# Checking the number of unique entries in Category column

transactions4['Category'].nunique()

111

### *Takeaways*

>**Category and description can help define the type of purchases and their categories so I will keep them both**

>**I will rename the columns and clean up Description. Then set date to a datetime type and an index column**

In [25]:
renamed = {
    'Date': 'date',
    'Description': 'description',
    'Amount': 'amount',
    'Transaction Type': 'transaction_type',
    'Category': 'category'
}

In [26]:
transactions5 = transactions4.rename(columns=renamed)
transactions5.head()

,date,description,amount,transaction_type,category
0,6/29/2022,DEPOSIT@MOBILE,206.25,credit,Transfer
1,6/28/2022,"MATTHEWS VET SERVICES, P",13.20,debit,Veterinary
2,6/28/2022,STARBUCKS STORE 24306,5.57,debit,Coffee Shops
3,6/28/2022,PAYPAL *HULU,14.16,debit,Television
4,6/28/2022,H-E-B #618,20.56,debit,Groceries


In [40]:
transactions4['Category'].unique()

array(['Transfer', 'Veterinary', 'Coffee Shops', 'Television',
       'Groceries', 'Shipping', 'Entertainment', 'Interest Income',
       'Restaurants', 'Gas & Fuel', 'Vacation', 'Shopping', 'Fast Food',
       'Charity', 'Travel', 'Business Services', 'Income',
       'Auto Insurance', 'Utilities', 'Alcohol & Bars',
       'Credit Card Payment', 'Newspapers & Magazines', 'Paycheck',
       'Parking', 'Internet', 'Clothing', 'Cash & ATM',
       'Electronics & Software', 'Air Travel', 'Pharmacy', 'Doctor',
       'Mortgage & Rent', 'Health & Fitness', 'Check',
       'Pet Food & Supplies', 'Books & Supplies', 'Uncategorized',
       'Furnishings', 'Gifts & Donations', 'Life Insurance',
       'Auto Payment', 'Rental Car & Taxi', 'Books', 'Laundry', 'Gift',
       'Ride Share', 'Service & Parts', 'Mobile Phone', 'Federal Tax',
       'Taxes', 'Education', 'Misc Expenses', 'Hotel',
       'Bills & Utilities', 'Service Fee', 'ATM Fee', 'Fees & Charges',
       'Tuition', 'Personal Care', 

In [44]:
transactions4.Category == 'Gym'

0       False
1       False
2       False
3       False
4       False
        ...  
7867    False
7868    False
7869    False
7870    False
7871    False
Name: Category, Length: 7797, dtype: bool

In [46]:
transactions4[transactions4.Category == 'Health & Fitness']

,Date,Description,Amount,Transaction Type,Category
115,5/23/2022,BMS-Medical Ctr SA04,113.69,debit,Health & Fitness
767,9/29/2020,NSHORE EH O P PHARM 847-570-2200 IL,1.42,debit,Health & Fitness
3571,6/12/2017,Emmaus Ministries,50.00,debit,Health & Fitness
3700,5/05/2017,Passport Health Anderson,420.00,debit,Health & Fitness
4010,2/13/2017,Emmaus Ministries,51.50,debit,Health & Fitness
4772,7/20/2016,Check,18.00,debit,Health & Fitness
5329,1/30/2016,Amazon,104.98,debit,Health & Fitness
5754,8/13/2015,W Morse,42.25,debit,Health & Fitness
5827,7/17/2015,W Morse,42.25,debit,Health & Fitness
5966,3/22/2015,Resurrection,100.00,debit,Health & Fitness


In [ ]:
transactions4.Category == 'Gym'].str.replace('Health & Fitness')

In [47]:
transactions4[transactions4.Category == 'Education']

,Date,Description,Amount,Transaction Type,Category
288,1/06/2022,CASH APP*JACOBO 8774174551 CA,100.00,debit,Education
332,11/19/2021,CASH APP*RODRIGO DE 8774174551 CA,250.00,debit,Education
1003,1/09/2020,COURSEHORSE.COM,30.00,debit,Education
1211,10/31/2019,PARCHMENT TRANSCRIPTS,7.50,debit,Education
1392,8/25/2019,PARCHMENT TRANSCRIPTS,10.00,debit,Education
1674,3/24/2019,Peterson's,60.00,credit,Education
1933,11/01/2018,Peterson's,87.47,debit,Education
2450,5/06/2018,Peterson's,9.92,debit,Education
2494,4/22/2018,Peterson's,75.01,debit,Education
2631,3/16/2018,Peterson's,85.00,debit,Education


In [45]:
transactions4[transactions4.Category == 'Gym']

,Date,Description,Amount,Transaction Type,Category
796,3/17/2020,LES MILLS,12.99,debit,Gym
893,2/19/2020,LES MILLS,12.99,debit,Gym
973,1/19/2020,LES MILLS,12.99,debit,Gym
1073,12/19/2019,LES MILLS,12.99,debit,Gym
1160,11/19/2019,LES MILLS,12.99,debit,Gym
1243,10/20/2019,LES MILLS,12.99,debit,Gym
1321,9/19/2019,LES MILLS,12.99,debit,Gym
1410,8/19/2019,LES MILLS,12.99,debit,Gym
1495,7/19/2019,LES MILLS,12.99,debit,Gym
1610,4/19/2019,LES MILLS,12.99,debit,Gym


In [ ]:
transactions4.Category == 'Gym'

In [31]:
transactions5['description'] = transactions5['description'].str.strip()

In [32]:
transactions5.head()

,date,description,amount,transaction_type,category
0,6/29/2022,DEPOSIT@MOBILE,206.25,credit,Transfer
1,6/28/2022,"MATTHEWS VET SERVICES, P",13.20,debit,Veterinary
2,6/28/2022,STARBUCKS STORE 24306,5.57,debit,Coffee Shops
3,6/28/2022,PAYPAL *HULU,14.16,debit,Television
4,6/28/2022,H-E-B #618,20.56,debit,Groceries


In [36]:
transactions5['description'] = transactions5['description'].str.replace('[#,@,&,*, ,]', '_').str.lower()

In [37]:
transactions5

,date,description,amount,transaction_type,category
0,6/29/2022,deposit_mobile,206.25,credit,Transfer
1,6/28/2022,matthews_vet_services__p,13.20,debit,Veterinary
2,6/28/2022,starbucks_store_24306,5.57,debit,Coffee Shops
3,6/28/2022,paypal__hulu,14.16,debit,Television
4,6/28/2022,h-e-b__618,20.56,debit,Groceries
...,...,...,...,...,...
7867,8/07/2010,ultra_foods_qps,17.84,debit,Food & Dining
7868,8/07/2010,pennys_noodle_shop,26.42,debit,Restaurants
7869,8/05/2010,amazon_music,5.00,debit,Music
7870,8/05/2010,calling_card,20.00,debit,Gift


In [38]:
transactions5['description'] = transactions5['description'].str.replace('__', '_')

In [39]:
transactions5

,date,description,amount,transaction_type,category
0,6/29/2022,deposit_mobile,206.25,credit,Transfer
1,6/28/2022,matthews_vet_services_p,13.20,debit,Veterinary
2,6/28/2022,starbucks_store_24306,5.57,debit,Coffee Shops
3,6/28/2022,paypal_hulu,14.16,debit,Television
4,6/28/2022,h-e-b_618,20.56,debit,Groceries
...,...,...,...,...,...
7867,8/07/2010,ultra_foods_qps,17.84,debit,Food & Dining
7868,8/07/2010,pennys_noodle_shop,26.42,debit,Restaurants
7869,8/05/2010,amazon_music,5.00,debit,Music
7870,8/05/2010,calling_card,20.00,debit,Gift


In [ ]:
# Setting the date column to a datetime type

transactions4.date = pd.to_datetime(transactions4.date)

In [ ]:
transactions5 = transactions4.set_index('date').sort_index()

In [ ]:
transactions5.head()

# Explore